In [ ]:
# | default_exp classes.DomoGroup


In [ ]:
# | exporti
from dataclasses import dataclass, field
from typing import List

import httpx
import asyncio

from fastcore.basics import patch_to


# from ..utils.Base import Base
# from ..utils.DictDot import DictDot

import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de
import domolibrary.utils.DictDot as util_dd

import domolibrary.routes.group as group_routes

# import domolibrary.routes.publish as publish_routes

import domolibrary.classes.DomoUser as dmu


# Group Membership

In [ ]:
# | export
class UpdateGroupMembership(de.DomoError):
    def __init__(member_name, group_name, domo_instance):
        super().__init__(domo_instance=domo_instance,
                         message=f"unable to add {member_name} to {group_name}")


class GroupMembership:
    _add_group_ls: list[str]
    _remove_group_ls: list[str]
    
    _add_owners_ls: list[str]
    _remove_owner_ls: list[str]

    _current_group_ls: list[str]
    _current_owner_ls: list[str] 

    group = None

    def __init__(self, group):
        self.group = group

        self._add_group_ls = []
        self._remove_group_ls = []
    
        self._add_owners_ls = []
        self._remove_owner_ls = []

        self._current_group_ls = []
        self._current_owner_ls = []

    
    def _add_group_member(self, member):
        import domolibrary.classes.DomoUser as dmu

        if not isinstance(member, DomoGroup) or isinstance(member, dmu.DomoUser):
            raise UpdateGroupMembership(domo_instance=self.group.auth.domo_instance,
                                        group_name=self.group.name,
                                        member_name=member.name)

        if member not in self._add_group_ls:
            self._add_group_ls.append(member)
    
    def _remove_group_member(self, member):
        import domolibrary.classes.DomoUser as dmu

        if not isinstance(member, DomoGroup) or isinstance(member, dmu.DomoUser):
            raise UpdateGroupMembership(domo_instance=self.group.auth.domo_instance,
                                        group_name=self.group.name,
                                        member_name=member.name)

        if member not in self._remove_group_ls:
            self._remove_group_ls.append(member)
    
    def _add_owner_member(self, member):
        import domolibrary.classes.DomoUser as dmu

        if not isinstance(member, DomoGroup) or isinstance(member, dmu.DomoUser):
            raise UpdateGroupMembership(domo_instance=self.group.auth.domo_instance,
                                        group_name=self.group.name,
                                        member_name=member.name)

        if member not in self._add_owners_ls:
            self._add_owners_ls.append(member)

    def _remove_owner_member(self, member):
        import domolibrary.classes.DomoUser as dmu

        if not isinstance(member, DomoGroup) or isinstance(member, dmu.DomoUser):
            raise UpdateGroupMembership(domo_instance=self.group.auth.domo_instance,
                                        group_name=self.group.name,
                                        member_name=member.name)

        if member not in self._remove_owner_ls:
            self._remove_owner_ls.append(member)

        
    # add
    # remove
    # set

In [ ]:
#| export
@patch_to(GroupMembership)
async def get_owners(
    self: GroupMembership,
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    import domolibrary.classes.DomoUser as dmu

    auth = auth or self.group.auth

    res = await group_routes.get_group_owners(group_id=self.group.id, auth=self.group.auth)

    if return_raw:
        return res

    owners = []
    
    group_ids = [obj.get('id') for obj in res.response if obj.get('type') == 'GROUP']
    if group_ids:
        domo_groups = await asyncio.gather(* [DomoGroup.get_by_id(group_id=group_id, auth=auth) for group_id in group_ids])
        self._current_owner_ls += domo_groups
    
    user_ids = [obj.get('id') for obj in res.response if obj.get('type') == 'USER']
    if user_ids:
        domo_users = await dmu.DomoUsers.by_id(user_ids=user_ids, auth=auth, only_allow_one = False)
        self._current_owner_ls += domo_users


    return self._current_owner_ls
    # return domo_users


In [ ]:
@patch_to(GroupMembership)
async def get_membership(
    self: GroupMembership,
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    import domolibrary.classes.DomoUser as dmu

    auth = auth or self.group.auth

    res = await group_routes.get_group_membership(group_id=self.group.id, auth=self.group.auth)

    if return_raw:
        return res

    domo_users = [dmu.DomoUser._from_search_json(user_obj = obj, auth = auth) for obj in res.response]

    self._current_group_ls = domo_users

    return domo_users


In [ ]:
# | exporti

@patch_to(GroupMembership)
async def set_membership(
    self: GroupMembership,
    user_ls : list[dmu.DomoUser],
    auth: dmda.DomoAuth = None,
    return_raw: bool = False,
    debug_api: bool = False, session: httpx.AsyncClient = None,
):

    auth = auth or self.group.auth

    await self.get_membership(auth = auth)

    self._current_owner_ls

    # add_user_arr = [str(uid) for uid in user_ids_list]

    # remove_user_arr = [
    #     str(uid) for uid in self.group_members_id_ls if str(uid) not in add_user_arr
    # ]

    # body = group_routes.generate_body_update_group_membership(
    #     group_id=self.group.id,
    #     add_user_arr=add_user_arr,
    #     remove_user_arr=remove_user_arr,
    # )

    # return await self._update_group_access(
    #     auth=auth, body=body, debug_api=debug_api, session=session, return_raw = return_raw
    # )

In [ ]:
# | exporti
@patch_to(GroupMembership)
async def alter_owner(self: GroupMembership, auth: dmda.DomoAuth,
                      add_owner_ids_ls=None,
                      remove_owner_ids_ls=None,
                      debug_api: bool = False,
                      session: httpx.AsyncClient = None
                      ):

    body = group_routes.generate_body_update_group_membership(group_id=self.group.id,
                                                              add_owner_user_arr=add_owner_ids_ls,
                                                              remove_owner_user_arr=remove_owner_ids_ls)

    return await self._update_group_access(auth=auth, body=body, log_results=log_results, debug_api=debug)


In [ ]:

#     async def alter_membership(self, auth: dmda.DomoAuth, add_user_ids_list, remove_users_ids_list,
#                                log_results: bool = False, debug_api: bool = False):

#         body = group_routes.generate_body_update_group_membership(group_id=self.group.id,
#                                                                   add_user_arr=add_user_ids_list,
#                                                                   remove_user_arr=remove_users_ids_list)

#         return await self._update_group_access(auth=auth, body=body, log_results=log_results, debug_api=debug)

# Domo Group

In [ ]:
# | export
@dataclass
class DomoGroup:
    auth: dmda.DomoAuth = field(repr=False, default=None)
    id: str = None
    name: str = None
    type: str = None
    description: str = None
    group_members_id_ls: list[str] = field(repr=False, default_factory=list)
    owner_members_id_ls: list[str] = field(repr=False, default_factory=list)
    
    group_members_ls: list[dict] = field(repr=False, default_factory=list)
    owner_members_ls : list[dict]= field(repr = False, default_factory = list)

    def __post_init__(self):
        # self.domo_instance = self.domo_instance or auth.domo_instance
        self.Membership = GroupMembership(self)

    @classmethod
    def _from_group_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.id,
            name=dd.name,
            description=dd.description,
            type=dd.type or dd.groupType,
            group_members_id_ls=dd.userIds,
            owner_members_ls=dd.owners,
        )

    @classmethod
    def _from_grouplist_json(cls, auth: dmda.DomoAuth, json_obj):
        dd = json_obj

        if not isinstance(json_obj, util_dd.DictDot):
            dd = util_dd.DictDot(json_obj)

        return cls(
            auth=auth,
            id=dd.groupId,
            name=dd.name,
            description=dd.description,
            type=dd.groupType,
            owner_members_ls = dd.owners,
            owner_members_id_ls = [owner.id for owner in dd.owners],
            
            group_members_ls=dd.groupMembers,
            group_members_id_ls= [member.id for member in dd.members]
        )


    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth) -> List[dict]:
        domo_groups = [
            DomoGroup._from_group_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

        return domo_groups

## Membership
### Get and SEARCH

#### sample implementation of get_owners

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group = DomoGroup(id=1781661643, auth=token_auth)

domo_group = GroupMembership(group=group)

await domo_group.get_owners()

# domo_group._current_owner_ls


#### sample implementation of get_membership

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group = DomoGroup( id =  1781661643, auth= token_auth )

domo_group = GroupMembership( group = group)

await domo_group.get_membership()

domo_group._current_group_ls


#### sample implementation of Group Membership
##### set_membership

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

membership = [{'type': 'USER', 'id': 1141078945},
              {'type': 'USER', 'id': 1725780094},
              {'type': 'USER', 'id': 861444808},
              {'type': 'USER', 'id': 1802840904}]

domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(
    f"validate membership {', '.join([str(id) for id in domo_group.Membership.group_members_id_ls])}")

await domo_group.Membership.set_membership(
    add_members_ls=membership
)


In [ ]:
# | exporti
@patch_to(DomoGroup, cls_method=True)
async def get_by_id(
    cls,
    auth: dmda.DomoAuth,
    group_id: str,
    return_raw: bool = False,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.get_group_by_id(
        auth=auth, group_id=group_id, debug_api=debug_api, session=session
    )
    if return_raw:
        return res

    if res.status == 200:
        return cls._from_group_json(auth=auth, json_obj=res.response)


#### sample implementation of get_by_id


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

res = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id, return_raw= False)

res

##### alter_owners

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_id = 2146122228

add_membership = [1141078945, 1725780094, 861444808, 1802840904]

domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

print(f"validate membership {', '.join([str(id) for id in domo_group.Membership.group_members_id_ls])}")

await domo_group.Membership.set_membership(
    user_ids_list= membership, return_raw = True
)


In [ ]:
# import os

# token_auth = dmda.DomoTokenAuth(
#     domo_instance="domo-community",
#     domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
# )

# group_id = 2146122228

# membership = [1141078945, 1725780094, 861444808, 1802840904]

# domo_group = await DomoGroup.get_by_id(auth=token_auth, group_id=group_id)

# print(
#     f"validate membership {', '.join([str(id) for id in domo_group.Membership.group_members_id_ls])}")

# await domo_group.Membership.set_membership(
#     user_ids_list=membership
# )


In [ ]:
# | exporti

@patch_to(DomoGroup, cls_method=True)
async def create_from_name(
    cls: DomoGroup,
    auth: dmda.DomoAuth,
    group_name: str = None,
    group_type: str = "open",  # use GroupType_Enum
    description: str = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.create_group(
        auth=auth,
        group_name=group_name,
        group_type=group_type,
        description=description,
        debug_api=debug_api,
    )

    domo_group = cls._from_group_json(auth=auth, json_obj=res.response)


#### sample implementation of create_from_name


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group 2"
try:
    await DomoGroup.create_from_name(group_name=group_name, auth=token_auth)

except group_routes.CreateGroup_Error as e:
    print(e)

In [ ]:
# | exporti


@patch_to(DomoGroup, cls_method=True)
async def search_by_name(
    cls,
    auth: dmda.DomoAuth,
    group_name: str,
    is_exact_match: bool = True,
    create_if_not_exist: bool = True,  # will set is_exact_match = True if enabled
    debug_api: bool = False,
    debug_prn: bool = False,
    session: httpx.AsyncClient = None,
):

    if create_if_not_exist:
        is_exact_match = True

    try:
        res = await group_routes.search_groups_by_name(
            auth=auth,
            search_name=group_name,
            debug_api=debug_api,
            is_exact_match=is_exact_match,
            session=session,
        )
        json_list = res.response

        if isinstance(json_list, list):
            return cls._groups_to_domo_group(json_list, auth)

        return cls._from_group_json(auth=auth, json_obj=json_list)

    except group_routes.SearchGroups_Error as e:
        print("create group")

        return await DomoGroup.create_from_name(
            auth=auth, group_name=group_name, debug_api=debug_api, session=session
        )


#### sample implementation of search_by_name


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group 2"

res = await DomoGroup.search_by_name(group_name=group_name, auth=token_auth)
print(res)

# Domo Groups

In [ ]:
# | export
class DomoGroups:
    def __init__(self):
        pass

    @staticmethod
    def _groups_to_domo_group(json_list, auth: dmda.DomoAuth):

        return [
            DomoGroup._from_group_json(auth=auth, json_obj=json_obj)
            for json_obj in json_list
        ]

In [ ]:
# | export
@patch_to(DomoGroups, cls_method=True)
async def get_all_groups(
    cls: DomoGroups,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):

    res = await group_routes.get_all_groups(
        auth=auth, debug_api=debug_api, session=session
    )

    if len(res.response) > 0:
        json_list = res.response

        return cls._groups_to_domo_group(json_list=json_list, auth=auth)

    else:
        return []


#### sample implementation of get_all_groups


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_groups = await DomoGroups.get_all_groups(auth=token_auth)
domo_groups[0:5]


# CRUD DomoGroups

In [ ]:
@patch_to(DomoGroups, cls_method=True)
async def upsert_group(cls: DomoGroups,
                       auth: dmda.DomoAuth,
                       group_name: str = None,
                       description: str = None,
                       group_id: str = None,
                       debug_api: bool = False,
                       debug_prn: bool = False,
                       session: httpx.AsyncClient = None):

    try:
        domo_group = None
        if group_id:
            domo_group = await DomoGroup.get_by_id(auth=auth, group_id=group_id, return_raw=False)

        if not domo_group and group_name:
            domo_goup = await DomoGroup.search_by_name(group_name=group_name, auth=auth)

        if not domo_group:
            raise user_routes. SearchGroups_Error(status=200,
                                                  message=f"{group_id or group_name} - not found",
                                                  domo_instance=auth.domo_instance,
                                                  function_name='upsert_groups')
        #     user_property_ls = []
        #     if display_name:
        #         user_property_ls.append(user_routes.UserProperty(
        #             user_routes.UserProperty_Type.display_name, display_name))

        #     if role_id:
        #         user_property_ls.append(user_routes.UserProperty(
        #             user_routes.UserProperty_Type.role_id, role_id))

        #     return await user_routes.update_user(
        #         user_id=domo_user.id,
        #         user_property_ls=user_property_ls,
        #         auth=auth,
        #         debug_api=debug_api
        #     )

        # if not role_id:
        #     raise CreateUser_MissingRole(
        #         domo_instance=auth.domo_instance, email_address=email_address)

        # return await cls.create_user(auth=auth,
        #                              display_name=display_name or f"{email_address} - via dl {dt.date.today()}",
        #                              email_address=email_address,
        #                              role_id=role_id,
        #                              debug_api=debug_api,
        #                              session=session)
    except:
        print("create User")


#### sample implementation of upsert_groups

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

group_name = "Test Group 2"

await DomoGroups.upsert_groups(auth=token_auth, group_name = group_name
)

